In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:

df = spark.table("workspace.bronze.erp_px_cat_g1v2")

In [0]:
df.display()

#Trimming

In [0]:

for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

In [0]:

df = df.withColumn(
    "maintenance",
        when(upper(col("maintenance")) == "YES", lit(True))
     .when(upper(col("maintenance")) == "NO", lit(False))
     .otherwise(None)
)


In [0]:
df.display()

In [0]:
RENAME_MAP = {
    "id": "category_id",
    "cat": "category",
    "subcat": "subcategory",
    "maintenance": "maintenance_flag"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_product_category")
     

In [0]:
%sql
SELECT * FROM workspace.silver.erp_product_category LIMIT 10